In [29]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

# Load Data



In [30]:
Data = pd.read_csv('ModuleReviews.csv', encoding = "ISO-8859-1" )

In [31]:
Data1 = pd.read_csv('All Reviews.csv', encoding = "ISO-8859-1" )

In [32]:
Data1.shape

(3616, 1)

In [33]:
Data1.duplicated().value_counts()

True     1853
False    1763
dtype: int64

In [34]:
data2= Data1.drop_duplicates()

In [35]:
data2.to_csv('All Reviews1.csv')

In [36]:
Data1 = pd.read_csv('All Reviews1.csv', encoding = "ISO-8859-1" )

In [37]:
Data1

,Unnamed: 0,Review
0,0,Update: Now that the tech issues have been add...
1,1,Changing my rating since the problems are fixe...
2,2,App worked pretty much as expected. Could have...
3,3,Highly recommend when you travel with Air Cana...
4,4,The app went from very useful to absolutely us...
...,...,...
1758,3602,"Terrible to find the flight, information onli..."
1759,3604,Worst App again.... Automatically shuts down. ...
1760,3609,My flight information just dissapears during l...
1761,3613,"This app just randomly loses your data, and cr..."


In [38]:
Data['Module'].value_counts()

User Experience        312
My trips page          110
Application Crashes     65
Update                  64
Flight Status           50
Booking Page            49
Login Issues            45
Boarding                30
Check-in                30
Baggage page            16
Error Messages          14
Payment Page            13
User Data issue         11
Maps                     6
Navigation               4
Seat selection           4
Calender                 2
Tracking                 2
Search page              1
Name: Module, dtype: int64

In [39]:
Data['Module'].isna().value_counts()

False    828
True       2
Name: Module, dtype: int64

In [40]:
Data['Module'] = Data['Module'].fillna(Data['Module'].mode()[0])

In [41]:
Data['Module'].isna().value_counts()

False    830
Name: Module, dtype: int64

In [42]:
Data.Module.replace({"Tracking":"Flight Status", "Search page":"Booking Page"},inplace=True)

In [43]:
Data['Module'].value_counts()

User Experience        314
My trips page          110
Application Crashes     65
Update                  64
Flight Status           52
Booking Page            50
Login Issues            45
Check-in                30
Boarding                30
Baggage page            16
Error Messages          14
Payment Page            13
User Data issue         11
Maps                     6
Navigation               4
Seat selection           4
Calender                 2
Name: Module, dtype: int64

In [44]:
Data.Module
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Data['labelled']= label_encoder.fit_transform(Data.Module)

In [45]:
# build train and test datasets
reviews = Data['REVIEW'].values
module = Data['labelled'].values
#creating the classification model using the classification algo
from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_module, test_module = train_test_split(reviews, module, test_size=0.2, random_state=42,stratify=module)


In [46]:
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

In [47]:
#function defined to remove html tags from data

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text
#function defined to remove unicode data
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [48]:
# lower case and remove special characters\whitespaces
def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
#         doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        #doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
    return norm_docs

In [49]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    #doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [50]:
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)


100%|██████████| 166/166 [00:00<00:00, 23775.12it/s]


In [51]:
# build TFIDF features on train reviews
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(use_idf=True, min_df=5, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(norm_train_reviews)

In [52]:
print('TFIDF model:> Train features shape:', tv_train_features.shape)

TFIDF model:> Train features shape: (664, 1491)


# Extract CV Features

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

stop_words = stop_words
cv = CountVectorizer(ngram_range=(1, 2), min_df=0.001, max_df=0.99, stop_words=stop_words)
cv_matrix = cv.fit_transform(norm_train_reviews)
cv_matrix.shape

(664, 10554)

# K Means

In [54]:

from sklearn.cluster import KMeans

NUM_CLUSTERS = 6
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(cv_matrix)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10000,
       n_clusters=6, n_init=100, n_jobs=None, precompute_distances='auto',
       random_state=42, tol=0.0001, verbose=0)

In [55]:
from collections import Counter

Counter(km.labels_)

Counter({5: 501, 2: 155, 4: 2, 3: 2, 0: 2, 1: 2})

In [61]:
kmeans_cluster = km.labels_

In [62]:
Data['kmeans_cluster'] = kmeans_cluster

ValueError: Length of values does not match length of index

In [57]:
Data['kmeans_cluster']

KeyError: 'kmeans_cluster'

In [ ]:

cat_cluster = (Data[['REVIEW', 'kmeans_cluster', 'Module']]
                  .sort_values(by=['kmeans_cluster'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))


In [ ]:
cat_cluster = cat_cluster.copy(deep=True)

In [ ]:
cat_cluster

In [ ]:
feature_names = cv.get_feature_names()
topn_features = 30
ordered_centroids = km.cluster_centers_.argsort()[:, ::-1]

In [ ]:
ordered_centroids

In [ ]:
#Getting Key Features for each Cluster
#Getting Module for Each Cluster

In [ ]:
for cluster_num in range(NUM_CLUSTERS):
    key_features = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    modules = cat_cluster[cat_cluster['kmeans_cluster'] == cluster_num]['Module'].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Key Features:', key_features)
    print('Impacted Module', modules)
    print('-'*80)

# Clustering with Embedding

In [ ]:
#pip install gensim

In [ ]:
from gensim.models import FastText

In [ ]:
tokenized_docs = [doc.split() for doc in norm_corpus]


In [ ]:
ft_model = FastText(tokenized_docs, size=100, window=20, min_count=2, workers=4, sg=1, iter=10)

In [ ]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)


In [ ]:
doc_vecs_ft

In [ ]:
doc_vecs_ft.shape

In [ ]:
NUM_CLUSTERS = 6
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(doc_vecs_ft)

In [ ]:
Counter(km.labels_)

In [ ]:
Data['kmeans_cluster'] = km.labels_

In [ ]:
cat_cluster = (Data[['REVIEW', 'kmeans_cluster', 'Module']]
                  .sort_values(by=['kmeans_cluster'], 
                               ascending=False)
                  .groupby('kmeans_cluster').head(20))


In [ ]:
cat_cluster = cat_cluster.copy(deep=True)

In [ ]:
for cluster_num in range(NUM_CLUSTERS):
    key_features = [feature_names[index] 
                        for index in ordered_centroids[cluster_num, :topn_features]]
    modules = cat_cluster[cat_cluster['kmeans_cluster'] == cluster_num]['Module'].values.tolist()
    print('CLUSTER #'+str(cluster_num+1))
    print('Key Features:', key_features)
    print('Impacted Module', modules)
    print('-'*80)

# Bag of Words Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

In [ ]:

# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

# Bag of N-Grams Model

In [ ]:

# you can set the n-gram range to 1,2 to get unigrams as well as bigrams
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

# TF-IDF

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

# Clustering using similarity

In [ ]:

from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2', 
                         'Distance', 'Cluster Size'], dtype='object')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8, 3))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Data point')
plt.ylabel('Distance')
dendrogram(Z)
plt.axhline(y=1.0, c='k', ls='--', lw=0.5)

In [ ]:
from scipy.cluster.hierarchy import fcluster
max_dist = 5.0

cluster_labels = fcluster(Z, max_dist, criterion='distance')
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
pd.concat([Data, cluster_labels], axis=1)